In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate

chat = ChatOpenAI(temperature=0.1,
                  # GPT 응답 과정을 보이게 하는 과정
                  # callback : LLM에 일어나는 events를 감지하는 방법
                  streaming= True,
                  callbacks=[StreamingStdOutCallbackHandler(),],
                  
                  )
# PromptTemplate를 사용하는 이유 : prompttemplate를 디스크에 저장하고 load하기 위함
# FewShot : 더 나은 대답을 위해 모델들에게 예제를 준다
# ex) 고객지원 봇을 만들 경우 이미 고객과의 대화에서 입력한 답변을 가져와 더 나은 답변을 하도록 도움을 줄 수 있다.

In [2]:
examples = [
      {
            "question": "What do you know about France?",
            "answer": """
            Here is what I know:
            Capital: Paris
            Language: French
            Food: Wine and Cheese
            Currency: Euro
            """,
      },
      {
            "question": "What do you know about Italy?",
            "answer": """
            I know this:
            Capital: Rome
            Language: Italian
            Food: Pizza and Pasta
            Currency: Euro
            """,
      },
      {
            "question": "What do you know about Greece?",
            "answer": """
            I know this:
            Capital: Athens
            Language: Greek
            Food: Souvlaki and Feta Cheese
            Currency: Euro
            """,
      },
]

In [3]:
# example을 사용하기 위한 형식 지정
example_template = """
      Human: {question}
      AI: {answer}
"""
example_prompt = PromptTemplate.from_template(example_template)

# 위와 동일
# example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

prompt = FewShotPromptTemplate(
      example_prompt = example_prompt,
      examples=examples,
      # 위 과정으로 랭체인이 알아서 각각의 예제들을 prompt를 사용하여 형식화함
      # 사용자 질문이 어떻게 보여줄지 지정
      # suffix : 형식화된 모든 예제 마지막에 나오는 내용
      suffix = "Human: What do you know about {country}?",
      input_variables = ["country"]
)

# 결과를 보면 위 예제와 같이 질문이 잘 작성됨을 알 수 있음
prompt.format(country = "Germany")

'\n      Human: What do you know about France?\n      AI: \n            Here is what I know:\n            Capital: Paris\n            Language: French\n            Food: Wine and Cheese\n            Currency: Euro\n            \n\n\n\n      Human: What do you know about Italy?\n      AI: \n            I know this:\n            Capital: Rome\n            Language: Italian\n            Food: Pizza and Pasta\n            Currency: Euro\n            \n\n\n\n      Human: What do you know about Greece?\n      AI: \n            I know this:\n            Capital: Athens\n            Language: Greek\n            Food: Souvlaki and Feta Cheese\n            Currency: Euro\n            \n\n\nHuman: What do you know about Germany?'

In [6]:
chain = prompt | chat

chain.invoke({"country" : "Korea"})

AI: 
I know this:
Capital: Seoul
Language: Korean
Food: Kimchi and Bibimbap
Currency: South Korean Won

AIMessageChunk(content='AI: \nI know this:\nCapital: Seoul\nLanguage: Korean\nFood: Kimchi and Bibimbap\nCurrency: South Korean Won')